In [ ]:
#! /usr/bin/env python

# PonyGE2
# Copyright (c) 2017 Michael Fenton, James McDermott,
#                    David Fagan, Stefan Forstenlechner,
#                    and Erik Hemberg
# Hereby licensed under the GNU GPL v3.
""" Python GE implementation """

from utilities.algorithm.general import check_python_version

check_python_version()
import matplotlib
matplotlib.use('Qt5Agg')
%matplotlib inline
import matplotlib.pyplot as plt
from stats.stats import get_stats
from algorithm.parameters import params, set_params
import sys
import time
import numpy as np
np.set_printoptions(linewidth=150)
import random
import os
import pickle

from utilities.stats import trackers


In [ ]:
# Print a specific element from the test_results array
seed_index = 0
hyperparam_combination_index = 0
print("Test result for seed index", seed_index, "and hyperparameter combination index", hyperparam_combination_index, ":", test_results[seed_index, hyperparam_combination_index])

# Print a specific element (list) from the test_evolution array
generation_index = 2
print("Test evolution for seed index", seed_index, "hyperparameter combination index", hyperparam_combination_index, "and generation index", generation_index, ":", test_evolution[seed_index, hyperparam_combination_index, generation_index])

In [ ]:
from itertools import product

def find_hyperparam_combination_index(desired_hyperparam_values, hyperparameters):
    # Generate all combinations of hyperparameter values
    hyperparam_combinations = list(product(*[hyperparam['list'] for hyperparam in hyperparameters]))
    
    for i, combination in enumerate(hyperparam_combinations):
        if all([desired_hyperparam_values[hyperparameters[j]['name']] == value for j, value in enumerate(combination)]):
            return i
    return -1  # Return -1 if the desired hyperparameter combination is not found

# Example usage
desired_hyperparam_values = {
    "MUTATION_PROBABILITY": 0.2,
    "ANOTHER_HYPERPARAMETER": 0.5
}

hyperparam_combination_index = find_hyperparam_combination_index(desired_hyperparam_values, hyperparameters)
print("Hyperparameter combination index:", hyperparam_combination_index)

In [ ]:
file_path = f"./results_data/results_data_fold_1AAAA.pkl"

# Load the data
with open(file_path, 'rb') as f:
    loaded_data = pickle.load(f)

# Access the loaded data
hyperparameters = loaded_data['hyperparameters']
seeds = loaded_data['seeds']
test_results = loaded_data['test_results']
train_results = loaded_data['train_results']
test_evolution = loaded_data['test_evolution']
train_evolution = loaded_data['train_evolution']

# Print hyperparameters
print("The analyzed hyperparameters are:")
for hyperparam in hyperparameters:
    print(hyperparam['name'], hyperparam['list'])

print("Seeds:", seeds)
print("Test results shape:", test_results.shape)
print("Train results shape:", train_results.shape)
print("Test evolution shape:", test_evolution.shape)
print("Train evolution shape:", train_evolution.shape)

In [ ]:
import numpy as np
from itertools import product

def reshape_test_results(test_results, seeds, hyperparameters):
    # Calculate the new shape for the test_results array
    new_shape = (len(seeds),) + tuple(len(hyperparam['list']) for hyperparam in hyperparameters)

    # Initialize an empty multidimensional array with the new shape
    reshaped_test_results = np.zeros(new_shape)

    # Create an iterable for all possible combinations of hyperparameter indices
    hyperparam_indices_combinations = product(*(range(len(hyperparam['list'])) for hyperparam in hyperparameters))

    # Iterate through the original test_results array
    for seed_index in range(len(seeds)):
        for hyperparam_indices in hyperparam_indices_combinations:
            # Calculate the index in the original test_results array
            original_index = sum(hyperparam_index * np.prod([len(hyperparameters[j]['list']) for j in range(i + 1, len(hyperparameters))]) for i, hyperparam_index in enumerate(hyperparam_indices))

            # Assign the result to the reshaped_test_results array using the calculated indices
            reshaped_test_results[(seed_index, *hyperparam_indices)] = test_results[seed_index, original_index]

    return reshaped_test_results

# Example usage
# Assuming you have a (5, 9) shaped array called test_results
reshaped_test_results = reshape_test_results(test_results, seeds, hyperparameters)
print("Reshaped test results:", reshaped_test_results)

In [ ]:
print(test_evolution[0][2])

In [ ]:
for i in range(5):
    file_path = f"./results_data/results_data_fold_{str(i+1)}.pkl"

    with open(file_path, 'rb') as f:
        loaded_data = pickle.load(f)

    # Access the loaded data
    hyperparam = loaded_data['hyperparam']
    hyperparam_list = hyperparam['list']
    seeds = loaded_data['seeds']
    test_results = loaded_data['test_results']
    train_results = loaded_data['train_results']
    test_evolution = loaded_data['test_evolution']
    train_evolution = loaded_data['train_evolution']

    avg_train_evoltion = np.mean(train_evolution, axis = 0)
    avg_test_evolution = np.mean(test_evolution, axis = 0)

    print("VALUES FOR FOLD",i+1)
    for j in range(test_evolution.shape[1]):
        
        print("SEED is", seeds[j])
        print("MUT_Value", hyperparam_list[j])
        #plt.plot(train_evolution[0][j], label = "Train Performances")
        #plt.plot(test_evolution[0][j], label = "Test Performances")
        avg_train_evoltion = np.mean(train_evolution, axis=0)
        avg_test_evolution = np.mean(test_evolution, axis = 0)
        plt.plot(avg_train_evoltion[j], label = "Train Performances")
        plt.plot(avg_test_evolution[j], label = "Test Performances")
        plt.legend()
        
        plt.xlabel("Generations")
        plt.ylabel("Accuracy")

        plt.show()


In [ ]:
matplotlib.use('Qt5Agg')
%matplotlib inline


# Generate mutation values
mean = np.mean(test_results, axis = 0)
std = np.std(test_results, axis = 0)

# Plot mean and standard deviation
plt.plot(hyperparam_list, mean, label="Mean")
plt.fill_between(hyperparam_list, mean - std, mean + std, alpha=0.3, label="Std")

# Set axis labels and title
plt.xlabel("Mutation Values", fontsize=16)
plt.ylabel("Fitness Values", fontsize=16)
plt.title("Mean and Standard Deviation", fontsize=16)

# Set xticks to display actual values of mutation_values rounded to 2 decimal places
rounded_x = np.round(hyperparam_list, 2)
plt.xticks(hyperparam_list, rounded_x, fontsize = 12)

# Add legend and display the plot
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming test_results is a 2D numpy array with shape (num_seeds, num_hyperparams)

# Calculate the average performance across seeds (axis 0)
avg_test_results = np.mean(test_results, axis=0)


# Set up the box plot
sns.set(style='whitegrid')
plt.figure(figsize=(10, 6))

# Create the box plot
sns.boxplot(data=test_results)

# Set plot labels and title
plt.xlabel('Hyperparameter Combination')
plt.ylabel('Performance')
plt.title('Box Plot of Test Results for Different Hyperparameter Combinations')

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assuming avg_test_results is a 1D numpy array with shape (num_hyperparams,)
# and hyperparam_values is a 1D numpy array with the hyperparameter values

# Create a scatter plot
plt.scatter(hyperparam_list, avg_test_results)

# Set plot labels and title
plt.xlabel('Hyperparameter')
plt.ylabel('Performance')
plt.title('Scatter Plot of Test Results for Different Hyperparameter Values')

# Show the plot
plt.show()


In [ ]:
# Create a line plot
plt.plot(hyperparam_list, avg_test_results, marker='o')

# Set plot labels and title
plt.xlabel('Hyperparameter')
plt.ylabel('Performance')
plt.title('Line Plot of Test Results for Different Hyperparameter Values')

# Show the plot
plt.show()


In [ ]:

print(train_evolution.shape)
agg_train_evolution = np.mean(train_evolution, axis=0)
print(agg_train_evolution.shape)

print(agg_train_evolution[0])


plt.plot(agg_train_evolution[1])
plt.show()